In [6]:
import pandas as pd
import enum
import os
from typing import List
from typing import Optional
from sqlalchemy import create_engine, select
from sqlalchemy import ForeignKey
from sqlalchemy import String, Integer
from sqlalchemy.orm import DeclarativeBase
from sqlalchemy.orm import Mapped
from sqlalchemy.orm import Session
from sqlalchemy.orm import mapped_column
from sqlalchemy.orm import relationship
import logging

logger = logging.getLogger("analysis")
logging.basicConfig(
    format="%(asctime)s %(levelname)s: %(message)s",
    level=logging.ERROR,
    datefmt="%H:%M:%S",
)


In [5]:
df_apps = pd.read_csv("data/java_github_top20.csv")


def map_repo_to_project_name(repo_name) -> str:
    match_filter = df_apps["clone_url"].str.endswith(f"{repo_name}.git")
    matched = df_apps[match_filter]
    matched_as_list = list(matched["name"])
    if matched.shape[0] == 0:
        logger.error(f'No project matched "{repo_name}"')
        return ""
    if matched.shape[0] > 1:
        logger.warning("Matched more than one project")
        return matched_as_list
    return matched_as_list[0]


def transform_path_to_repo_name(path: str) -> str:
    path = path.split("/")[-1]  # get the portion that ends the url in .git
    path = path.split(".git")[0]  # removes .git
    return path


def map_project_name_to_repo(project) -> str:
    match_filter = df_apps["name"] == project
    matched = df_apps[match_filter]
    matched_as_list = list(matched["clone_url"])
    if matched.shape[0] == 0:
        logger.error(f'No repo matched "{project}"')
        return ""
    if matched.shape[0] > 1:
        logger.warning("Matched more than one repo")
        return [
            transform_path_to_repo_name(repo) for repo in matched_as_list["clone_url"]
        ]
    return transform_path_to_repo_name(matched_as_list[0])


## Create Models

In [10]:
class Base(DeclarativeBase):
    pass


class Project(Base):
    __tablename__ = "projects"
    id: Mapped[int] = mapped_column(primary_key=True, autoincrement=True)
    name: Mapped[str] = mapped_column(String(), nullable=False)
    description: Mapped[str] = mapped_column(String(), nullable=True)
    package: Mapped[str] = mapped_column(String(), nullable=False)
    category: Mapped[str] = mapped_column(String(), nullable=False)
    aoc_reports: Mapped[List["AoCReport"]] = relationship(back_populates="project")


class AoCReport(Base):
    __tablename__ = "aoc_reports"
    id: Mapped[int] = mapped_column(primary_key=True, autoincrement=True)
    project_id: Mapped[int] = mapped_column(ForeignKey("projects.id"))
    project: Mapped["Project"] = relationship(back_populates="aoc_reports")
    line: Mapped[int] = mapped_column(Integer())
    snippet: Mapped[str] = mapped_column(String(), nullable=False)
    class_: Mapped[str] = mapped_column(String(), nullable=False, name="class")
    aoc: Mapped[str] = mapped_column(String(), nullable=False)
    path: Mapped[str] = mapped_column(String(), nullable=True)


### Create connection engine

In [11]:
engine = create_engine("sqlite+pysqlite:///aocs_dataset.db")
Base.metadata.create_all(engine)


### Persist projects

In [11]:
df_projects = pd.read_csv("data/java_github_top20.csv")
projects: List[Project] = []
for _, row in df_projects.iterrows():
    projects.append(
        Project(
            name=row["name"],
            description=row["summary"],
            package=row["package"],
            category=row["category"],
        )
    )
with Session(engine) as session:
    session.add_all(projects)
    session.commit()


### Persist AOCs

In [12]:
def query_project(project_name):
    with Session(engine) as session:
        stmt = select(Project).where(Project.name.like(project_name))
        return session.scalar(stmt)


In [18]:
BOHR_REPORTS_PATH = "./bohr_reports"
_, _, files = next(os.walk(BOHR_REPORTS_PATH))
consolidated_csvs = [file for file in files if file.endswith("_consolidated.csv")]
for csv_file_name in consolidated_csvs:
    repo_name = csv_file_name.split("_consolidated.csv")[0]
    project_name = map_repo_to_project_name(repo_name)
    df_aocs = pd.read_csv(f"{BOHR_REPORTS_PATH}/{csv_file_name}")
    df_aocs = df_aocs.fillna("")
    aocs = []
    for _, row in df_aocs.iterrows():
        project = query_project(project_name)
        aocs.append(
            AoCReport(
                project_id = project.id,
                line = row["Line"],
                snippet = row["Snippet"],
                class_ = row["Class"],
                aoc = row["Atom"],
                path = row["Path"]
            )
        )
    with Session(engine) as session:
        session.add_all(aocs)
        session.commit()
